In [1]:
import torchvision.datasets as datasets
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings('ignore')

2024-05-17 11:53:10.242579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 11:53:13.311608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
writer = SummaryWriter('logs')

In [3]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),           
    transforms.Normalize(            
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225]   
    )
])

train_data = datasets.Imagenette(root='data', split='train', download=False, transform=transform)
test_data = datasets.Imagenette(root='data', split='val', download=False, transform=transform)


In [4]:
train_data.classes

[('tench', 'Tinca tinca'),
 ('English springer', 'English springer spaniel'),
 ('cassette player',),
 ('chain saw', 'chainsaw'),
 ('church', 'church building'),
 ('French horn', 'horn'),
 ('garbage truck', 'dustcart'),
 ('gas pump', 'gasoline pump', 'petrol pump', 'island dispenser'),
 ('golf ball',),
 ('parachute', 'chute')]

In [5]:
class VGG11(nn.Module):

    def __init__(self):
        super(VGG11, self).__init__()
        self.conv3_1 = nn.Conv2d(3, 64, (3,3), stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(64, 128, (3,3), stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(128, 256, (3,3), stride=1, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, (3,3), stride=1, padding=1)
        self.conv3_5 = nn.Conv2d(256, 512, (3,3), stride=1, padding=1)
        self.conv3_6 = nn.Conv2d(512, 512, (3,3), stride=1, padding=1)
        self.conv3_7 = nn.Conv2d(512, 512, (3,3), stride=1, padding=1)
        self.conv3_8 = nn.Conv2d(512, 512, (3,3), stride=1, padding=1)

        self.fc1 = nn.Linear(7*7*512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, len(train_data.classes))
        
    def forward(self, X):
        out = F.relu(self.conv3_1(X))
        out = F.max_pool2d(out, (2,2), stride=2) # 112
        out = F.relu(self.conv3_2(out))          # <-- Use 'out' as input
        out = F.max_pool2d(out, (2,2), stride=2) # 56
        out = F.relu(self.conv3_3(out))          # <-- Use 'out' as input
        out = F.relu(self.conv3_4(out))
        out = F.max_pool2d(out, (2,2), stride=2) # 28
        out = F.relu(self.conv3_5(out))          # <-- Use 'out' as input
        out = F.relu(self.conv3_6(out))
        out = F.max_pool2d(out, (2,2), stride=2) # 14
        out = F.relu(self.conv3_7(out))          # <-- Use 'out' as input
        out = F.relu(self.conv3_8(out))
        out = F.max_pool2d(out, (2,2), stride=2) # 7
        out = out.view(out.size(0), -1)
        out = F.dropout2d(out, 0.5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model = VGG11().to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [9]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(test_data, batch_size=16, shuffle=True)


In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total_correct += (torch.max(outputs, 1)[1] == labels).sum().item()
        total_samples += labels.size(0)
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] train loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            # Log the running loss to TensorBoard
            writer.add_scalar('training loss', running_loss / 100, epoch * len(train_loader) + i)
            running_loss = 0.0

    # Calculate training accuracy
    train_accuracy = total_correct / total_samples

    # Validation
    model.eval()
    val_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            total_correct += (torch.max(outputs, 1)[1] == labels).sum().item()
            total_samples += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = total_correct / total_samples

    print('[%d] val loss: %.3f, train accuracy: %.3f, val accuracy: %.3f' % (epoch + 1, val_loss, train_accuracy, val_accuracy))
    # Log validation loss and accuracy to TensorBoard
    writer.add_scalar('validation loss', val_loss, epoch)
    writer.add_scalar('training accuracy', train_accuracy, epoch)
    writer.add_scalar('validation accuracy', val_accuracy, epoch)

[1,   100] train loss: 2.304
[1,   200] train loss: 2.305
[1,   300] train loss: 2.304
[1,   400] train loss: 2.304
[1,   500] train loss: 2.305
[1] val loss: 2.304, train accuracy: 0.092, val accuracy: 0.099
[2,   100] train loss: 2.304



KeyboardInterrupt



In [ ]:
writer.close()